In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

from xgboost import XGBClassifier
import joblib


In [ ]:
pip install sklearn

In [ ]:
df = pd.read_csv(
    r"C:\Users\ADMIN\OneDrive\Desktop\E-commerce Product Delivery Prediction\E_Commerce.csv"
)
df = df.sample(frac=0.5, random_state=42)

print(df.shape)
df.info()
print(df.isnull().sum())


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
sns.boxplot(y=df["Discount_offered"])
plt.title("Box Plot - Discount Offered")
plt.show()


In [ ]:

df["discount_ratio"] = df["Discount_offered"] / (df["Cost_of_the_Product"] + 1)


df["weight_category"] = pd.cut(
    df["Weight_in_gms"],
    bins=[0, 1000, 3000, 10000],
    labels=[0, 1, 2]
)


df["weight_category"] = df["weight_category"].astype(int)


In [ ]:
"""We engineered new features to improve model performance. The discount ratio captures the relative impact of discounts,
 while weight category groups products into logistical weight ranges.
The categorical weight feature was converted into numeric format to ensure model compatibility."""

In [ ]:
le = LabelEncoder()

categorical_cols = [
    "Warehouse_block",
    "Mode_of_Shipment",
    "Product_importance",
    "Gender"
]

for col in categorical_cols:
    df[col] = le.fit_transform(df[col])


In [ ]:
plt.figure(figsize=(7,4))
sns.countplot(
    x="weight_category",
    hue="Reached.on.Time_Y.N",
    data=df
)
plt.title("Weight Category vs Delivery Status")
plt.xlabel("Weight Category (0=Light, 1=Medium, 2=Heavy)")
plt.ylabel("Order Count")
plt.legend(title="Delivery Status")
plt.show()


In [ ]:
X = df.drop(["Reached.on.Time_Y.N", "ID"], axis=1)
y = df["Reached.on.Time_Y.N"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)
       

In [15]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


NameError: name 'StandardScaler' is not defined

In [16]:
"""Stratify ensures that both training and testing datasets have the same proportion 
of classes as the original dataset, preventing bias during model training."""

'Stratify ensures that both training and testing datasets have the same proportion \nof classes as the original dataset, preventing bias during model training.'

In [17]:
def evaluate_model(model, X_tr, X_te, y_tr, y_te, name):
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_te)

    acc = accuracy_score(y_te, y_pred)

    print("\n" + "="*40)
    print(name)
    print("Accuracy:", acc)
    print("="*40)
    print(classification_report(y_te, y_pred))

    return acc


In [ ]:
lr = LogisticRegression(
    max_iter=1000,
    class_weight="balanced"
)

knn = KNeighborsClassifier(
    n_neighbors=7,
    weights="distance"
)

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=10,
    min_samples_split=10,
    class_weight="balanced",
    random_state=42
)

xgb = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    tree_method="hist",
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05
)


In [ ]:
lr_acc = evaluate_model(
    lr, X_train_scaled, X_test_scaled,
    y_train, y_test,
    "Logistic Regression"
)

knn_acc = evaluate_model(
    knn, X_train_scaled, X_test_scaled,
    y_train, y_test,
    "KNN"
)

rf_acc = evaluate_model(
    rf, X_train, X_test,
    y_train, y_test,
    "Random Forest"
)

xgb_acc = evaluate_model(
    xgb, X_train, X_test,
    y_train, y_test,
    "XGBoost"
)


In [ ]:
print("\n FINAL MODEL ACCURACY COMPARISON")
print(f"Logistic Regression Accuracy : {lr_acc:.4f}")
print(f"KNN Accuracy                : {knn_acc:.4f}")
print(f"Random Forest Accuracy      : {rf_acc:.4f}")
print(f"XGBoost Accuracy            : {xgb_acc:.4f}")


In [ ]:
voting = VotingClassifier(
    estimators=[
        ("rf", rf),
        ("xgb", xgb)
    ],
    voting="soft"
)

voting.fit(X_train, y_train)

y_pred_vote = voting.predict(X_test)

print("\nVoting Classifier (RF + XGB)")
print("Accuracy:", accuracy_score(y_test, y_pred_vote))
print(classification_report(y_test, y_pred_vote))


In [ ]:
"""A Voting Classifier combines predictions from multiple models.
In this project, Random Forest and XGBoost were combined using soft voting to improve prediction accuracy
and reduce individual model errors."""

In [11]:
joblib.dump(voting, "delivery_model.pkl")
joblib.dump(scaler, "scaler.pkl")

print(" Model and scaler saved successfully")


NameError: name 'joblib' is not defined

In [ ]:
feature_order = X.columns.tolist()

joblib.dump(feature_order, "feature_order.pkl")


In [ ]:
"""Despite applying advanced ensemble methods, accuracy stabilized around 67% due to inherent data limitations and overlapping features.
 This behavior is expected in real-world logistics prediction problems."""

In [ ]:
# ============================================================================
# DETAILED MODEL EVALUATION FOR EACH MACHINE LEARNING MODEL
# ============================================================================

import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score,
    confusion_matrix, 
    classification_report,
    roc_auc_score,
    roc_curve
)
import matplotlib.pyplot as plt
import seaborn as sns

# ============================================================================
# 1. LOGISTIC REGRESSION EVALUATION
# ============================================================================
print("=" * 80)
print("LOGISTIC REGRESSION - DETAILED EVALUATION")
print("=" * 80)

# Predictions
y_pred_lr = lr.predict(X_test_scaled)
y_proba_lr = lr.predict_proba(X_test_scaled)[:, 1]

# Metrics
lr_accuracy = accuracy_score(y_test, y_pred_lr)
lr_precision = precision_score(y_test, y_pred_lr)
lr_recall = recall_score(y_test, y_pred_lr)
lr_f1 = f1_score(y_test, y_pred_lr)
lr_auc = roc_auc_score(y_test, y_proba_lr)

print(f"\nAccuracy:  {lr_accuracy:.4f} ({lr_accuracy*100:.2f}%)")
print(f"Precision: {lr_precision:.4f}")
print(f"Recall:    {lr_recall:.4f}")
print(f"F1-Score:  {lr_f1:.4f}")
print(f"ROC-AUC:   {lr_auc:.4f}")

print("\nConfusion Matrix:")
cm_lr = confusion_matrix(y_test, y_pred_lr)
print(cm_lr)
print(f"\nTrue Negatives:  {cm_lr[0,0]}")
print(f"False Positives: {cm_lr[0,1]}")
print(f"False Negatives: {cm_lr[1,0]}")
print(f"True Positives:  {cm_lr[1,1]}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr, target_names=['Delayed', 'On Time']))

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Confusion Matrix
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Delayed', 'On Time'],
            yticklabels=['Delayed', 'On Time'])
axes[0].set_title('Logistic Regression\nConfusion Matrix', fontweight='bold')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')

# Metrics Bar Chart
metrics_lr = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC']
values_lr = [lr_accuracy, lr_precision, lr_recall, lr_f1, lr_auc]
axes[1].bar(metrics_lr, values_lr, color='steelblue', edgecolor='black', alpha=0.8)
axes[1].set_title('Logistic Regression\nPerformance Metrics', fontweight='bold')
axes[1].set_ylabel('Score')
axes[1].set_ylim([0, 1])
for i, v in enumerate(values_lr):
    axes[1].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')

# ROC Curve
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_proba_lr)
axes[2].plot(fpr_lr, tpr_lr, color='steelblue', lw=2, label=f'AUC = {lr_auc:.3f}')
axes[2].plot([0, 1], [0, 1], 'k--', lw=1)
axes[2].set_title('Logistic Regression\nROC Curve', fontweight='bold')
axes[2].set_xlabel('False Positive Rate')
axes[2].set_ylabel('True Positive Rate')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('eval_logistic_regression.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# 2. K-NEAREST NEIGHBORS (KNN) EVALUATION
# ============================================================================
print("\n" + "=" * 80)
print("K-NEAREST NEIGHBORS (KNN) - DETAILED EVALUATION")
print("=" * 80)

# Predictions
y_pred_knn = knn.predict(X_test_scaled)
y_proba_knn = knn.predict_proba(X_test_scaled)[:, 1]

# Metrics
knn_accuracy = accuracy_score(y_test, y_pred_knn)
knn_precision = precision_score(y_test, y_pred_knn)
knn_recall = recall_score(y_test, y_pred_knn)
knn_f1 = f1_score(y_test, y_pred_knn)
knn_auc = roc_auc_score(y_test, y_proba_knn)

print(f"\nAccuracy:  {knn_accuracy:.4f} ({knn_accuracy*100:.2f}%)")
print(f"Precision: {knn_precision:.4f}")
print(f"Recall:    {knn_recall:.4f}")
print(f"F1-Score:  {knn_f1:.4f}")
print(f"ROC-AUC:   {knn_auc:.4f}")

print("\nConfusion Matrix:")
cm_knn = confusion_matrix(y_test, y_pred_knn)
print(cm_knn)
print(f"\nTrue Negatives:  {cm_knn[0,0]}")
print(f"False Positives: {cm_knn[0,1]}")
print(f"False Negatives: {cm_knn[1,0]}")
print(f"True Positives:  {cm_knn[1,1]}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_knn, target_names=['Delayed', 'On Time']))

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Confusion Matrix
sns.heatmap(cm_knn, annot=True, fmt='d', cmap='Greens', ax=axes[0],
            xticklabels=['Delayed', 'On Time'],
            yticklabels=['Delayed', 'On Time'])
axes[0].set_title('KNN\nConfusion Matrix', fontweight='bold')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')

# Metrics Bar Chart
metrics_knn = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC']
values_knn = [knn_accuracy, knn_precision, knn_recall, knn_f1, knn_auc]
axes[1].bar(metrics_knn, values_knn, color='mediumseagreen', edgecolor='black', alpha=0.8)
axes[1].set_title('KNN\nPerformance Metrics', fontweight='bold')
axes[1].set_ylabel('Score')
axes[1].set_ylim([0, 1])
for i, v in enumerate(values_knn):
    axes[1].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')

# ROC Curve
fpr_knn, tpr_knn, _ = roc_curve(y_test, y_proba_knn)
axes[2].plot(fpr_knn, tpr_knn, color='mediumseagreen', lw=2, label=f'AUC = {knn_auc:.3f}')
axes[2].plot([0, 1], [0, 1], 'k--', lw=1)
axes[2].set_title('KNN\nROC Curve', fontweight='bold')
axes[2].set_xlabel('False Positive Rate')
axes[2].set_ylabel('True Positive Rate')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('eval_knn.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# 3. RANDOM FOREST EVALUATION
# ============================================================================
print("\n" + "=" * 80)
print("RANDOM FOREST - DETAILED EVALUATION")
print("=" * 80)

# Predictions
y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:, 1]

# Metrics
rf_accuracy = accuracy_score(y_test, y_pred_rf)
rf_precision = precision_score(y_test, y_pred_rf)
rf_recall = recall_score(y_test, y_pred_rf)
rf_f1 = f1_score(y_test, y_pred_rf)
rf_auc = roc_auc_score(y_test, y_proba_rf)

print(f"\nAccuracy:  {rf_accuracy:.4f} ({rf_accuracy*100:.2f}%)")
print(f"Precision: {rf_precision:.4f}")
print(f"Recall:    {rf_recall:.4f}")
print(f"F1-Score:  {rf_f1:.4f}")
print(f"ROC-AUC:   {rf_auc:.4f}")

print("\nConfusion Matrix:")
cm_rf = confusion_matrix(y_test, y_pred_rf)
print(cm_rf)
print(f"\nTrue Negatives:  {cm_rf[0,0]}")
print(f"False Positives: {cm_rf[0,1]}")
print(f"False Negatives: {cm_rf[1,0]}")
print(f"True Positives:  {cm_rf[1,1]}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf, target_names=['Delayed', 'On Time']))

# Feature Importance
print("\nTop 10 Most Important Features:")
feature_names = X.columns.tolist()
feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)
print(feature_importance.head(10).to_string(index=False))

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Confusion Matrix
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Oranges', ax=axes[0],
            xticklabels=['Delayed', 'On Time'],
            yticklabels=['Delayed', 'On Time'])
axes[0].set_title('Random Forest\nConfusion Matrix', fontweight='bold')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')

# Metrics Bar Chart
metrics_rf = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC']
values_rf = [rf_accuracy, rf_precision, rf_recall, rf_f1, rf_auc]
axes[1].bar(metrics_rf, values_rf, color='coral', edgecolor='black', alpha=0.8)
axes[1].set_title('Random Forest\nPerformance Metrics', fontweight='bold')
axes[1].set_ylabel('Score')
axes[1].set_ylim([0, 1])
for i, v in enumerate(values_rf):
    axes[1].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')

# ROC Curve
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_proba_rf)
axes[2].plot(fpr_rf, tpr_rf, color='coral', lw=2, label=f'AUC = {rf_auc:.3f}')
axes[2].plot([0, 1], [0, 1], 'k--', lw=1)
axes[2].set_title('Random Forest\nROC Curve', fontweight='bold')
axes[2].set_xlabel('False Positive Rate')
axes[2].set_ylabel('True Positive Rate')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('eval_random_forest.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# 4. XGBOOST EVALUATION
# ============================================================================
print("\n" + "=" * 80)
print("XGBOOST - DETAILED EVALUATION")
print("=" * 80)

# Predictions
y_pred_xgb = xgb.predict(X_test)
y_proba_xgb = xgb.predict_proba(X_test)[:, 1]

# Metrics
xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
xgb_precision = precision_score(y_test, y_pred_xgb)
xgb_recall = recall_score(y_test, y_pred_xgb)
xgb_f1 = f1_score(y_test, y_pred_xgb)
xgb_auc = roc_auc_score(y_test, y_proba_xgb)

print(f"\nAccuracy:  {xgb_accuracy:.4f} ({xgb_accuracy*100:.2f}%)")
print(f"Precision: {xgb_precision:.4f}")
print(f"Recall:    {xgb_recall:.4f}")
print(f"F1-Score:  {xgb_f1:.4f}")
print(f"ROC-AUC:   {xgb_auc:.4f}")

print("\nConfusion Matrix:")
cm_xgb = confusion_matrix(y_test, y_pred_xgb)
print(cm_xgb)
print(f"\nTrue Negatives:  {cm_xgb[0,0]}")
print(f"False Positives: {cm_xgb[0,1]}")
print(f"False Negatives: {cm_xgb[1,0]}")
print(f"True Positives:  {cm_xgb[1,1]}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_xgb, target_names=['Delayed', 'On Time']))

# Feature Importance
print("\nTop 10 Most Important Features:")
feature_importance_xgb = pd.DataFrame({
    'Feature': feature_names,
    'Importance': xgb.feature_importances_
}).sort_values('Importance', ascending=False)
print(feature_importance_xgb.head(10).to_string(index=False))

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Confusion Matrix
sns.heatmap(cm_xgb, annot=True, fmt='d', cmap='Purples', ax=axes[0],
            xticklabels=['Delayed', 'On Time'],
            yticklabels=['Delayed', 'On Time'])
axes[0].set_title('XGBoost\nConfusion Matrix', fontweight='bold')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')

# Metrics Bar Chart
metrics_xgb = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC']
values_xgb = [xgb_accuracy, xgb_precision, xgb_recall, xgb_f1, xgb_auc]
axes[1].bar(metrics_xgb, values_xgb, color='mediumpurple', edgecolor='black', alpha=0.8)
axes[1].set_title('XGBoost\nPerformance Metrics', fontweight='bold')
axes[1].set_ylabel('Score')
axes[1].set_ylim([0, 1])
for i, v in enumerate(values_xgb):
    axes[1].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')

# ROC Curve
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_proba_xgb)
axes[2].plot(fpr_xgb, tpr_xgb, color='mediumpurple', lw=2, label=f'AUC = {xgb_auc:.3f}')
axes[2].plot([0, 1], [0, 1], 'k--', lw=1)
axes[2].set_title('XGBoost\nROC Curve', fontweight='bold')
axes[2].set_xlabel('False Positive Rate')
axes[2].set_ylabel('True Positive Rate')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('eval_xgboost.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# 5. VOTING CLASSIFIER EVALUATION
# ============================================================================
print("\n" + "=" * 80)
print("VOTING CLASSIFIER (RF + XGB) - DETAILED EVALUATION")
print("=" * 80)

# Predictions
y_pred_vote = voting.predict(X_test)
y_proba_vote = voting.predict_proba(X_test)[:, 1]

# Metrics
vote_accuracy = accuracy_score(y_test, y_pred_vote)
vote_precision = precision_score(y_test, y_pred_vote)
vote_recall = recall_score(y_test, y_pred_vote)
vote_f1 = f1_score(y_test, y_pred_vote)
vote_auc = roc_auc_score(y_test, y_proba_vote)

print(f"\nAccuracy:  {vote_accuracy:.4f} ({vote_accuracy*100:.2f}%)")
print(f"Precision: {vote_precision:.4f}")
print(f"Recall:    {vote_recall:.4f}")
print(f"F1-Score:  {vote_f1:.4f}")
print(f"ROC-AUC:   {vote_auc:.4f}")

print("\nConfusion Matrix:")
cm_vote = confusion_matrix(y_test, y_pred_vote)
print(cm_vote)
print(f"\nTrue Negatives:  {cm_vote[0,0]}")
print(f"False Positives: {cm_vote[0,1]}")
print(f"False Negatives: {cm_vote[1,0]}")
print(f"True Positives:  {cm_vote[1,1]}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_vote, target_names=['Delayed', 'On Time']))

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Confusion Matrix
sns.heatmap(cm_vote, annot=True, fmt='d', cmap='RdYlGn', ax=axes[0],
            xticklabels=['Delayed', 'On Time'],
            yticklabels=['Delayed', 'On Time'])
axes[0].set_title('Voting Classifier\nConfusion Matrix', fontweight='bold')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')

# Metrics Bar Chart
metrics_vote = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC']
values_vote = [vote_accuracy, vote_precision, vote_recall, vote_f1, vote_auc]
axes[1].bar(metrics_vote, values_vote, color='darkseagreen', edgecolor='black', alpha=0.8)
axes[1].set_title('Voting Classifier\nPerformance Metrics', fontweight='bold')
axes[1].set_ylabel('Score')
axes[1].set_ylim([0, 1])
for i, v in enumerate(values_vote):
    axes[1].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')

# ROC Curve
fpr_vote, tpr_vote, _ = roc_curve(y_test, y_proba_vote)
axes[2].plot(fpr_vote, tpr_vote, color='darkseagreen', lw=2, label=f'AUC = {vote_auc:.3f}')
axes[2].plot([0, 1], [0, 1], 'k--', lw=1)
axes[2].set_title('Voting Classifier\nROC Curve', fontweight='bold')
axes[2].set_xlabel('False Positive Rate')
axes[2].set_ylabel('True Positive Rate')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('eval_voting_classifier.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# 6. COMPREHENSIVE COMPARISON
# ============================================================================
print("\n" + "=" * 80)
print("COMPREHENSIVE MODEL COMPARISON")
print("=" * 80)

# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'Model': ['Logistic Regression', 'KNN', 'Random Forest', 'XGBoost', 'Voting Classifier'],
    'Accuracy': [lr_accuracy, knn_accuracy, rf_accuracy, xgb_accuracy, vote_accuracy],
    'Precision': [lr_precision, knn_precision, rf_precision, xgb_precision, vote_precision],
    'Recall': [lr_recall, knn_recall, rf_recall, xgb_recall, vote_recall],
    'F1-Score': [lr_f1, knn_f1, rf_f1, xgb_f1, vote_f1],
    'ROC-AUC': [lr_auc, knn_auc, rf_auc, xgb_auc, vote_auc]
})

print("\n" + comparison_df.to_string(index=False))

# Save to CSV
comparison_df.to_csv('model_comparison.csv', index=False)
print("\n✓ Comparison table saved as 'model_comparison.csv'")

# Visualization - All Metrics Comparison
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
colors_list = ['steelblue', 'mediumseagreen', 'coral', 'mediumpurple', 'darkseagreen']

for idx, metric in enumerate(metrics):
    axes[idx].bar(comparison_df['Model'], comparison_df[metric], 
                  color=colors_list, edgecolor='black', alpha=0.8)
    axes[idx].set_title(f'{metric} Comparison', fontweight='bold', fontsize=12)
    axes[idx].set_ylabel(metric)
    axes[idx].set_ylim([0, 1])
    axes[idx].tick_params(axis='x', rotation=45)
    axes[idx].grid(axis='y', alpha=0.3)
    
    # Add value labels
    for i, v in enumerate(comparison_df[metric]):
        axes[idx].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold', fontsize=9)

# Overall comparison
axes[5].axis('off')
best_model_idx = comparison_df['Accuracy'].idxmax()
best_model = comparison_df.loc[best_model_idx, 'Model']
best_accuracy = comparison_df.loc[best_model_idx, 'Accuracy']

axes[5].text(0.5, 0.7, 'BEST MODEL', ha='center', fontsize=20, fontweight='bold')
axes[5].text(0.5, 0.5, best_model, ha='center', fontsize=16, color='darkgreen')
axes[5].text(0.5, 0.3, f'Accuracy: {best_accuracy*100:.2f}%', ha='center', fontsize=14)

plt.tight_layout()
plt.savefig('eval_comprehensive_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "=" * 80)
print("✓ ALL MODEL EVALUATIONS COMPLETED!")
print("=" * 80)
print("\nSaved Files:")
print("  • eval_logistic_regression.png")
print("  • eval_knn.png")
print("  • eval_random_forest.png")
print("  • eval_xgboost.png")
print("  • eval_voting_classifier.png")
print("  • eval_comprehensive_comparison.png")
print("  • model_comparison.csv")
print("=" * 80)